<a href="https://colab.research.google.com/github/Nohgyumin0812/GlobalIndustryProxyTrading/blob/main/global_industry_proxytrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install gspread
!pip install finance-datareader


In [ ]:
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pytz
import FinanceDataReader as fdr
from statsmodels.tsa.stattools import coint
import numpy as np
import matplotlib.pyplot as plt
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import chardet
from time import sleep
from scipy.stats import spearmanr

In [ ]:
all_data = pd.read_csv('/content/drive/MyDrive/all_data.csv').set_index('Unnamed: 0')
local_timezone = pytz.timezone('Asia/Seoul')
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
def corr_coint_test(proxy, df):
  merged_df = all_data.reset_index().rename(columns = {'Unnamed: 0':'DATE'}).dropna(axis = 1)

  all_data_ex = pd.merge(merged_df, df, on='DATE', how = 'left').iloc[:-1,].dropna(axis = 0).reset_index(drop= True).set_index(keys = 'DATE')
  shape = int(all_data_ex.shape[0] / 2)
  all_data_ex = all_data_ex[shape:]
  tickers = all_data_ex.columns[1:-1]
  proxy_array = np.array(all_data_ex[proxy])
  print(proxy_array)
  corr_list = []
  for ticker in tickers:
    ticker_array = np.array(all_data_ex[ticker])
    corr_score, corr_pval = spearmanr(proxy_array, ticker_array)
    corr_list.append([ticker, corr_score])
  corr_list = sorted(corr_list, key = lambda x:x[1], reverse= True)[:5]
  return corr_list

In [ ]:
# 화학-에틸렌(워크시트2)
url = 'https://www.koreapds.com/wacs/wms/kpds_price.php?large_cd=22'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser',from_encoding='ISO-8859-1')
nafta = float(soup.find_all('span', class_='price')[0].text.strip())
ethy = float(soup.find_all('span', class_='price')[1].text.strip())

current_date = datetime.now(local_timezone).date().strftime("%Y-%m-%d")
cell_list = [current_date,ethy]

worksheet = gc.open('Global Industry Proxy_Trading').get_worksheet(1)
raw_data = worksheet.get_all_values()

df = pd.DataFrame(raw_data, columns=raw_data[0]).drop(0).reset_index(drop = True)
df.loc[:, ~df.columns.isin(['DATE'])] = df.loc[:, ~df.columns.isin(['DATE'])].astype(float)

if current_date == df.iloc[-1]['DATE']:
  df.iloc[-1] = cell_list
else:
  df.loc[df.shape[0]+2] = cell_list

worksheet.update([df.columns.values.tolist()] + df.values.tolist())  # 컬럼은 컬럼값만, 값들은 이중리스트로 df 변환"""

##코릴레이션 업뎃##
corr_list = corr_coint_test('ethy', df)
print(corr_list)
worksheet = gc.open('Global Industry Proxy_Trading').get_worksheet(0)
worksheet.update('L3:M7', corr_list)

/usr/local/lib/python3.10/dist-packages/bs4/__init__.py:226: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


[730.0 730.0 750.0 750.0 790.0 790.0 810.0 810.0 810.0 810.0 820.0 870.0
 870.0 870.0 870.0 870.0 855.0 850.0 850.0 850.0 850.0 850.0 850.0 850.0
 850.0]


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


[['018670', 0.868585214396406], ['082640', 0.8646487303066874], ['018290', 0.8520848956609609], ['001080', 0.8458299116242058], ['025950', 0.8404393531549181]]


{'spreadsheetId': '1haCnx_AugipQwMIfHKc2RC4tcQCMArUuXER4Ka0gNtc',
 'updatedRange': "'프록시 인덱스 정보 탭'!L3:M7",
 'updatedRows': 5,
 'updatedColumns': 2,
 'updatedCells': 10}

In [ ]:
worksheet = gc.open('Global Industry Proxy_Trading').get_worksheet(2) ## 납사

raw_data = worksheet.get_all_values()
current_date = datetime.now(local_timezone).date().strftime("%Y-%m-%d")
cell_list = [current_date,nafta]

df = pd.DataFrame(raw_data, columns=raw_data[0]).drop(0).reset_index(drop = True)
df.loc[:, ~df.columns.isin(['DATE'])] = df.loc[:, ~df.columns.isin(['DATE'])].astype(float)

if current_date == df.iloc[-1]['DATE']:
  df.iloc[-1] = cell_list
else:
  df.loc[df.shape[0]+2] = cell_list

worksheet.update([df.columns.values.tolist()] + df.values.tolist())  # 컬럼은 컬럼값만, 값들은 이중리스트로 df 변환"""
##코릴레이션 업뎃##
corr_list = corr_coint_test('nafta', df)
print(corr_list)
worksheet = gc.open('Global Industry Proxy_Trading').get_worksheet(0)
worksheet.update('L17:M21', corr_list)

[533.875 549.25 602.625 600.5 638.75 653.375 664.875 644.625 663.5 666.25
 685.5 712.75 718.75 647.75 657.0 687.25 657.5 657.5 630.25 625.75 625.75
 625.75 642.15 642.15]


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


[['011330', 0.8471592217662233], ['083310', 0.8454513087962421], ['224060', 0.7996080992722367], ['204620', 0.7984334193266261], ['205100', 0.7844550696539971]]


{'spreadsheetId': '1haCnx_AugipQwMIfHKc2RC4tcQCMArUuXER4Ka0gNtc',
 'updatedRange': "'프록시 인덱스 정보 탭'!L17:M21",
 'updatedRows': 5,
 'updatedColumns': 2,
 'updatedCells': 10}

In [ ]:
# DRAM(워크시트 4)

worksheet = gc.open('Global Industry Proxy_Trading').get_worksheet(3)

url = 'https://www.dramexchange.com/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

dram_name_list = soup.find_all('td', class_ = 'tab_title')
dram1_price = float(soup.find_all('td', class_='tab_tr_gray')[4].text.strip())
dram2_price = float(soup.find_all('td', class_='tab_tr_gray')[16].text.strip())

print(dram1_price) # session_average
print(dram2_price) # session_average

raw_data = worksheet.get_all_values()
current_date = datetime.now(local_timezone).date().strftime("%Y-%m-%d")
cell_list = [current_date, dram1_price, dram2_price]

df = pd.DataFrame(raw_data, columns=raw_data[0]).drop(0)
df.loc[:, ~df.columns.isin(['DATE'])] = df.loc[:, ~df.columns.isin(['DATE'])].astype(float)

if current_date == list(df.DATE)[-1]:
  df.iloc[-1] = cell_list
else:
  df.loc[df.shape[0]+1] = cell_list

worksheet.update([df.columns.values.tolist()] + df.values.tolist())  # 컬럼은 컬럼값만, 값들은 이중리스트로 df 변환

##코릴레이션 업뎃##
corr_list = corr_coint_test('DDR4 16Gb (1Gx16)3200(우)', df)
print(corr_list)
worksheet = gc.open('Global Industry Proxy_Trading').get_worksheet(0)
worksheet.update('L31:M35', corr_list)

3.506
1.603
[2.92 2.914 2.909 2.931 2.962 2.95 2.949 3.016 3.041 3.022 3.104 3.171
 3.197 3.2 3.26 3.386 3.42 3.446 3.475 3.483]


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


[['001720', 0.9404266016427048], ['000320', 0.939760101884157], ['001630', 0.9262606083373501], ['018260', 0.9233082706766916], ['000815', 0.9203007518796992]]


{'spreadsheetId': '1haCnx_AugipQwMIfHKc2RC4tcQCMArUuXER4Ka0gNtc',
 'updatedRange': "'프록시 인덱스 정보 탭'!L31:M35",
 'updatedRows': 5,
 'updatedColumns': 2,
 'updatedCells': 10}

In [ ]:
#2차전지-리튬 워크시트 5
url = 'https://www.kores.net/komis/price/mineralprice/minormetals/pricetrend/minorMetals.do?mnrl_pc_mc_seq=516'
response = requests.get(url, verify = False)
soup = BeautifulSoup(response.text, 'html.parser')


date = soup.find_all('td', class_ = 'fst')[-2].text.strip()
lit = float(soup.find('table').find_all('td')[-9].text.strip())

worksheet = gc.open('Global Industry Proxy_Trading').get_worksheet(4)
cell_list = [date, lit]
print(cell_list)
raw_data = worksheet.get_all_values()

df = pd.DataFrame(raw_data, columns=raw_data[0]).drop(0).reset_index(drop = True)
df.loc[:, ~df.columns.isin(['DATE'])] = df.loc[:, ~df.columns.isin(['DATE'])].astype(float)


if date == df.iloc[-1]['DATE']:
  df.iloc[-1] = cell_list
else:
  df.loc[df.shape[0]+2] = cell_list

worksheet.update([df.columns.values.tolist()] + df.values.tolist())  # 컬럼은 컬럼값만, 값들은 이중리스트로 df 변환"""

corr_list = corr_coint_test('lit', df)
print(corr_list)
worksheet = gc.open('Global Industry Proxy_Trading').get_worksheet(0)
worksheet.update('L45:M49', corr_list)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.kores.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


['2023-12-14', 88.5]
[304.5 304.5 302.5 302.5 302.5 302.5 302.5 302.5 302.5 302.5 302.5 302.5
 302.5 302.5 297.5 297.5 297.5 292.5 292.5 292.5 287.5 287.5 280.5 275.5
 270.5 262.5 257.5 257.5 257.5 257.5 257.5 257.5 252.5 247.5 247.5 247.5
 240.5 227.5 222.5 212.5 202.5 202.5 202.5 202.5 202.5 202.5 201.5 197.5
 195.5 195.5 195.5 195.5 192.5 192.5 192.5 186.5 180.5 175.5 172.5 166.5
 162.5 161.5 157.5 155.5 155.5 153.5 153.5 153.5 153.5 153.5 155.5 158.5
 162.5 165.5 165.5 163.5 162.5 158.5 157.5 159.5 157.5 155.5 155.5 154.5
 154.5 154.5 154.5 153.5 149.5 147.5 147.5 144.5 144.5 143.5 139.5 135.5
 131.5 131.5 129.5 126.5 123.5 120.5 109.5 100.5 95.5 90.5 90.5]


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


[['000300', 0.9809065927369612], ['073570', 0.9756360319616332], ['001070', 0.9610720989862586], ['005860', 0.9557308994723724], ['084670', 0.9500438364656772]]


{'spreadsheetId': '1haCnx_AugipQwMIfHKc2RC4tcQCMArUuXER4Ka0gNtc',
 'updatedRange': "'프록시 인덱스 정보 탭'!L45:M49",
 'updatedRows': 5,
 'updatedColumns': 2,
 'updatedCells': 10}

In [ ]:
#신조선가- 벌커 워크시트 6

url = 'https://www.kobc.or.kr/ebz/shippinginfo/sln/gridList.do?mId=0401000000'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', class_ = 'TbCommon')
worksheet = gc.open('Global Industry Proxy_Trading').get_worksheet(5)


date = table.find_all('td')[0].text.strip()
cape =  float(table.find_all('td')[1].text.strip())
kamsarmax =  float(table.find_all('td')[2].text.strip())
ultramax =  float(table.find_all('td')[3].text.strip())
handy =  float(table.find_all('td')[4].text.strip())

cell_list = [date, cape, kamsarmax, ultramax, handy]

raw_data = worksheet.get_all_values()

df = pd.DataFrame(raw_data, columns=raw_data[0]).drop(0).reset_index(drop = True)
df.loc[:, ~df.columns.isin(['DATE'])] = df.loc[:, ~df.columns.isin(['DATE'])].astype(float)


if date == df.iloc[-1]['DATE']:
  df.iloc[-1] = cell_list
else:
  df.loc[df.shape[0]+2] = cell_list

worksheet.update([df.columns.values.tolist()] + df.values.tolist())  # 컬럼은 컬럼값만, 값들은 이중리스트로 df 변환

corr_list = corr_coint_test('CAPE(180K)(우)', df)
print(corr_list)
worksheet = gc.open('Global Industry Proxy_Trading').get_worksheet(0)
worksheet.update('T3:U7', corr_list)

[62.96 62.77 62.73 62.53 62.8 62.57 62.75 62.55 62.41 62.55 62.54 62.71
 62.72 62.8 63.19 63.43 63.5 63.49 63.58 63.53 63.57 63.78 64.02 64.23]


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


[['006650', 0.8618664550233356], ['000325', 0.8559930591915259], ['001530', 0.8434102581251922], ['072870', 0.8272035016346602], ['012170', 0.8157494210846816]]


{'spreadsheetId': '1haCnx_AugipQwMIfHKc2RC4tcQCMArUuXER4Ka0gNtc',
 'updatedRange': "'프록시 인덱스 정보 탭'!T3:U7",
 'updatedRows': 5,
 'updatedColumns': 2,
 'updatedCells': 10}

In [ ]:
#신조선가- 탱커 워크시트 7


url = 'https://www.kobc.or.kr/ebz/shippinginfo/stn/gridList.do?mId=0401000000'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', class_ = 'TbCommon')
worksheet = gc.open('Global Industry Proxy_Trading').get_worksheet(6)


date = table.find_all('td')[0].text.strip()
vlcc =  float(table.find_all('td')[1].text.strip())
suezmax =  float(table.find_all('td')[2].text.strip())
aframax =  float(table.find_all('td')[3].text.strip())
lr2 =  float(table.find_all('td')[4].text.strip())
lr1 =  float(table.find_all('td')[5].text.strip())
mr =  float(table.find_all('td')[6].text.strip())

cell_list = [date, vlcc,suezmax,aframax,lr2,lr1,mr]

raw_data = worksheet.get_all_values()

df = pd.DataFrame(raw_data, columns=raw_data[0]).drop(0).reset_index(drop = True)
df.loc[:, ~df.columns.isin(['DATE'])] = df.loc[:, ~df.columns.isin(['DATE'])].astype(float)


if date == df.iloc[-1]['DATE']:
  df.iloc[-1] = cell_list
else:
  df.loc[df.shape[0]+2] = cell_list

df_tanker = df['VLCC(320K)(우)']
worksheet.update([df.columns.values.tolist()] + df.values.tolist())  # 컬럼은 컬럼값만, 값들은 이중리스트로 df 변환

corr_list = corr_coint_test('VLCC(320K)(우)', df)
print(corr_list)
worksheet = gc.open('Global Industry Proxy_Trading').get_worksheet(0)
worksheet.update('T17:U21', corr_list)

[122.0 124.0 124.0 123.0 124.0 124.0 123.0 124.0 124.0 124.0 123.0 123.0
 123.0 123.0 123.0 123.0 123.0 123.0 123.0 123.0 123.33 123.38 123.53
 123.53]


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


[['003490', 0.5735320136039815], ['001120', 0.5552524593164057], ['001460', 0.5458214196677194], ['073540', 0.5125770049061003], ['001430', 0.5063366721284454]]


{'spreadsheetId': '1haCnx_AugipQwMIfHKc2RC4tcQCMArUuXER4Ka0gNtc',
 'updatedRange': "'프록시 인덱스 정보 탭'!T17:U21",
 'updatedRows': 5,
 'updatedColumns': 2,
 'updatedCells': 10}

In [ ]:
#BDI-  발틱운임지수 워크시트 8


url = 'https://kr.investing.com/indices/baltic-dry-historical-data'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

bdi = float(soup.find_all('div', class_ = 'text-5xl/9 font-bold md:text-[42px] md:leading-[60px] text-[#232526]')[0].text.strip().replace(',', ''))
print(bdi)
worksheet = gc.open('Global Industry Proxy_Trading').get_worksheet(7)


date = datetime.now(local_timezone).date().strftime("%Y-%m-%d")

cell_list = [date, bdi]

raw_data = worksheet.get_all_values()

df = pd.DataFrame(raw_data, columns=raw_data[0]).drop(0).reset_index(drop = True)
df = df.apply(lambda x : x.str.replace(',' ,''))
df.loc[:, ~df.columns.isin(['DATE'])] = df.loc[:, ~df.columns.isin(['DATE'])].replace(',', ).astype(float)

print(df)
if current_date == list(df.DATE)[-1]:
  df.iloc[-1] = cell_list
else:
  df.loc[df.shape[0]+1] = cell_list

df_bdi = df
worksheet.update([df.columns.values.tolist()] + df.values.tolist())  # 컬럼은 컬럼값만, 값들은 이중리스트로 df 변환"""

corr_list = corr_coint_test('bdi', df)
print(corr_list)
worksheet = gc.open('Global Industry Proxy_Trading').get_worksheet(0)
worksheet.update('T31:U35', corr_list)

2348.0
           DATE     bdi
0    2023-01-03  1250.0
1    2023-01-04  1176.0
2    2023-01-05  1146.0
3    2023-01-06  1130.0
4    2023-01-09  1139.0
..          ...     ...
235  2023-12-07  2848.0
236  2023-12-09  2495.0
237  2023-12-10  2483.0
238  2023-12-16  2411.0
239  2023-12-17  2348.0

[240 rows x 2 columns]
[1216.0 1240.0 1233.0 1183.0 1138.0 1112.0 1091.0 1068.0 1044.0 994.0
 993.0 1009.0 1024.0 1032.0 1088.0 1103.0 1090.0 1073.0 1037.0 1004.0
 977.0 978.0 967.0 962.0 1067.0 1097.0 1110.0 1127.0 1150.0 1123.0 1128.0
 1136.0 1145.0 1142.0 1144.0 1137.0 1129.0 1135.0 1233.0 1247.0 1237.0
 1223.0 1194.0 1151.0 1110.0 1080.0 1107.0 1094.0 1086.0 1065.0 1083.0
 1063.0 1081.0 1141.0 1186.0 1209.0 1235.0 1290.0 1340.0 1381.0 1439.0
 1526.0 1584.0 1569.0 1593.0 1614.0 1694.0 1752.0 1778.0 1827.0 1929.0
 1983.0 1948.0 1935.0 1945.0 1972.0 2058.0 2105.0 2071.0 2046.0 2017.0
 1949.0 1832.0 1662.0 1563.0 1502.0 1459.0 1401.0 1385.0 1462.0 1523.0
 1559.0 1530.0 1598.0 1643.0 1655.0 1662.

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


[['003090', 0.9010933486964424], ['024110', 0.9003947081767426], ['018670', 0.8804475762142676], ['204210', 0.8703018660692291], ['024940', 0.8635591415104885]]


{'spreadsheetId': '1haCnx_AugipQwMIfHKc2RC4tcQCMArUuXER4Ka0gNtc',
 'updatedRange': "'프록시 인덱스 정보 탭'!T31:U35",
 'updatedRows': 5,
 'updatedColumns': 2,
 'updatedCells': 10}

In [ ]:
###coint test###
"""
all_data_ex= merged_df[120:]

all_data_ex = np.log(all_data_ex.apply(lambda x:x.astype(float)))
print(all_data_ex)

tickers = all_data_ex.columns[1:-1]
coint_list = []
i = 0
for ticker in tickers:
  i+=1
  coint_score = coint(all_data_ex['bdi'], all_data_ex[ticker])[0]
  coint_pval = coint(all_data_ex['bdi'], all_data_ex[ticker])[1]
  result = coint(all_data_ex['bdi'], all_data_ex[ticker])
  coint_list.append([ticker, coint_score, coint_pval])
coint_list = sorted(coint_list, key = lambda x:x[2], reverse= False)[:5]


print(coint_list)"""

"\nall_data_ex= merged_df[120:]\n\nall_data_ex = np.log(all_data_ex.apply(lambda x:x.astype(float)))\nprint(all_data_ex)\n\ntickers = all_data_ex.columns[1:-1]\ncoint_list = []\ni = 0\nfor ticker in tickers:\n  i+=1\n  coint_score = coint(all_data_ex['bdi'], all_data_ex[ticker])[0]\n  coint_pval = coint(all_data_ex['bdi'], all_data_ex[ticker])[1]\n  result = coint(all_data_ex['bdi'], all_data_ex[ticker])\n  coint_list.append([ticker, coint_score, coint_pval])\ncoint_list = sorted(coint_list, key = lambda x:x[2], reverse= False)[:5]\n\n\nprint(coint_list)"

In [ ]:
### 종목코드 가져오기##
"""import FinanceDataReader as fdr
all_data = pd.DataFrame()
stock_df = fdr.StockListing('KRX')

stock_df = stock_df[stock_df['Market']!='KONEX'][['Code', 'Name']]
stock_value = stock_df.values.tolist()
worksheet = gc.open('Global Industry Proxy_Trading').get_worksheet(0)
worksheet.update('W2', stock_value)"""

{'spreadsheetId': '1haCnx_AugipQwMIfHKc2RC4tcQCMArUuXER4Ka0gNtc',
 'updatedRange': "'프록시 인덱스 정보 탭'!W2:X2658",
 'updatedRows': 2657,
 'updatedColumns': 2,
 'updatedCells': 5314}

In [ ]:
# 백테스팅 가격 데이터셋 수집
"""
!pip install finance-datareader
from google.colab import drive
drive.mount('/content/drive')
start_date = '2023-01-01'
import pandas as pd
import FinanceDataReader as fdr
all_data = pd.DataFrame()
stock_df = fdr.StockListing('KRX')

stock_df = stock_df[stock_df['Market']!='KONEX']
symbols = list(stock_df['Code'])
i = 0
for symbol in symbols:
   i+=1
   print(i)
   data = fdr.DataReader(symbol = symbol, start = start_date)
   data['symbol'] = symbol
   all_data = pd.concat([all_data, data], axis = 0 )

all_data = all_data.pivot_table(index = 'Date', columns = 'symbol', values = 'Close')
all_data.index.name = None
all_data.columns.name = None
all_data.to_csv('/content/drive/MyDrive/all_data.csv')"""

MessageError: ignored